### Po co symulować chorobę zakaźną?


### Modelowanie agentowe

<strong>Modele agnentowe</strong> to dynamiczne modele symulacyjne, składające się ze struktury systemu, agentów wykonujących czynności i podejmujących decyzje oraz procesów zachodzących w systemie.

<strong>Trzy podstawowe komponenty:</strong>
* agenci
* powiązania między agengtami
* środowisko działania agentów

#### Co charakteryzuje agentów?
* Cechy wyrażone za pomocą danych
* Funkcje pozwalające podejmować działania na danych
* Funkcje określające, jakie czynności wykonać i w jakiej kolejności 
* Funkcje pozwalające na wykorzystanie posiadanych danych do podjęcia decyzji w przyszłości

#### Środowisko funkcjonowania agentów

Środowisko działania agentów to przestrzeń, w której agenci podejmują wszelkie akcje.

Atrybuty i zachowania agentów można zmieniać, a wywoływane tym następstwa są następnie analizowane. Dzięki temu modele agentowe są pomocne przy analizie problemów o różnych skalach przestrzennych, czasowych oraz na różnych poziomach organizacji. Model agentowy reprezentuje wybrany systemu oraz wyjaśnia, jak ten system funkcjonuje. Pozwala na przeprowadzanie eksperymentów, które mogą być wykonywane wielokrotnie, przy użyciu różnych parametrów, bez żadnej szkody dla badanego systemu. 
Przez indywidualne modelowanie zachowań agentów, model agentowy pozwala na ujęcie zróżnicowania elementów i obiektów, występujących w badanym systemie.

#### Co jest ważne przy tworzeniu modelu agentowego?
* Cel modelu
* Definicja agentów
* Definicja właściwości agentów
* Definicja działań podejmowanych przez agentów
* Definicja podejmowania decyzji przez agentów
* Określenie interwału czasowego każdego kolejnego kroku modelu
* Określenie czasu trwania symulacji
* Określenie sposobu weryfikacji i walidacji modelu


In [ ]:
using Agents , Random , InteractiveDynamics , CairoMakie 
using DrWatson: @dict


mutable struct Agent2 <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
    days_infected::Int # how many days have passed after infection
    status::Symbol # :S (susceptible), :I (infecious), :R (removed, immune), :Q (quarantine and infecious) 
    days_quarantine::Int
    hygiene::Float64
    carefull::Bool
end


function symulation(;
    infection_period = 24 ,
    detection_time = 14 ,
    reinfection_probability = 0.01,
    isolated = 0.2, # 0 is nobody is isolated , 1 everyone is isolated
    interaction_radius = 0.01,
    dt = 1,
    speed = 0.0012,
    death_rate = 0.044,
    N = 200,
    initial_infected = 5,
    seed = 1410,
    hygiene_min = 0.4,
    hygiene_max = 0.8,
    time_of_quarantine = 10,
    steps_per_day = 12,
    chance_to_go_quaratine = 0.5
    )

    infection_period *= steps_per_day
    detection_time *=  steps_per_day
    time_of_quarantine *= steps_per_day

    properties = @dict(
        infection_period,
        detection_time,
        reinfection_probability,
        death_rate,
        dt,
        interaction_radius,
        time_of_quarantine,
        speed,
        chance_to_go_quaratine,
        steps_per_day)

    space = ContinuousSpace((1,1), 0.02)
    model = ABM(Agent2,space, properties = properties, rng = MersenneTwister(seed))

    # Add agents to the model
    for id in 1:N
        pos = Tuple(rand(model.rng, 2))
        status = id > initial_infected ? :S : :I
        is_isolated = isolated * N >= id
        mass = is_isolated ? Inf : 1.0
        vel = is_isolated ? (0,0) : sincos(2pi* rand(model.rng,)) .* speed
        hygiene = (hygiene_max-hygiene_min)*rand(model.rng) + hygiene_min
        carefull = id > 0.1*N ? false : true
        add_agent!(pos,model,vel, mass, 0, status, 0, hygiene, carefull)
    end
    return model
end

function transmit!(a,b,reinfection,model)
    if  count(a.status == :I for a in (a,b)) in (0,2)
        return
    end
    
    infected, healthy = a.status == :I ? (a,b) : (b,a)
    rand(model.rng) < (infected.hygiene + healthy.hygiene)/2 && return 

   
    if healthy.status == :R
        if rand(model.rng) > reinfection 
            return
        end
    end

    healthy.status = :I
end

function avoid(a, model)
    if a.mass != Inf
        a.vel = sincos(2pi* rand(model.rng,)) .* model.speed
    end
end

function model_step!(model)
    r = model.interaction_radius
    r2 = 5*r
    for (a, b) in interacting_pairs(model, r, :nearest)
        transmit!(a,b,model.reinfection_probability,model)
        elastic_collision!(a, b, :mass)
    end
    for (a, b) in interacting_pairs(model, r2, :nearest)
        a.carefull == true  ? avoid(a, model) : continue  
    end

end

function quarantine!(a,model)
    if a.status == :I && a.days_infected >= model.detection_time && rand(model.rng) < model.chance_to_go_quaratine / model.steps_per_day
        a.status = :Q
        a.vel = (0,0)
        a.mass = Inf

    end
end

function quarantine_end!(a,model)
    if a.days_quarantine == model.time_of_quarantine
        a.days_quarantine = 0
        recover_or_die!(a,model) 
        a.vel = sincos(2pi* rand(model.rng,)) .* model.speed
        a.mass = 1.0
    end
end

function recover_or_die!(agent, model)
    if agent.days_infected >= model.infection_period
        if rand(model.rng) <= model.death_rate
            kill_agent!(agent, model)
        else 
            agent.status = :R
            agent.days_infected = 0
        end
    end
end

function agent_step!(agent,model)
    move_agent!(agent, model, model.dt)
    agent.status in (:I,:Q) ? agent.days_infected +=1 : agent.days_infected = 0
    agent.status == :Q ? agent.days_quarantine +=1 : agent.days_quarantine = 0
    recover_or_die!(agent, model)
    quarantine!(agent,model)
    quarantine_end!(agent,model)
end

### Model SIR

### Parametry użyte w symulacji
1. izolacja
2. higiena 
3. kwarantanna
4. szczepienia
5. gęstość zaludnienia
6. aktywność ludzka w ciągu dnia


In [1]:
using Agents , Random , InteractiveDynamics , CairoMakie 
using DrWatson: @dict
import Distributions: Categorical



mutable struct Agent2 <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
    days_infected::Int # how many days have passed after infection
    status::Symbol # :S (susceptible), :I (infecious), :R (removed, immune), :Q (quarantine and infecious) :V (vaccinated and removed)
    days_quarantine::Int
    hygiene::Float64
end

function if_susceptible(agent::Agent2)
    return agent.status == :S
end


function symulation(;
    infection_period = 24 ,
    detection_time = 14 ,
    reinfection_probability = 0.01,
    isolated = 0.2, # 0 is nobody is isolated , 1 eve ryone is isolated
    interaction_radius = 0.01,
    dt = 1,
    speed = 0.0012,
    death_rate = 0.044,
    N = 1000,
    initial_infected = 5,
    seed = 1410,
    hygiene_min = 0.4,
    hygiene_max = 0.8,
    steps_per_day = 12,
    chance_to_go_quarantine = 0.5,
    symulation_time = 0,
    vaccine_per_day = 2
    )

    infection_period *= steps_per_day
    detection_time *=  steps_per_day

    properties = @dict(
        infection_period,
        detection_time,
        reinfection_probability,
        death_rate,
        dt,
        interaction_radius,
        speed,
        chance_to_go_quarantine,
        steps_per_day,
        symulation_time,
        N,
        vaccine_per_day)

    space = ContinuousSpace((1,1), 0.02)
    model = ABM(Agent2,space, properties = properties, rng = MersenneTwister(seed))

    # Add agents to the model
   for idx in 1:N
        pos = Tuple(rand(model.rng, 2))
        status = idx > initial_infected ? :S : :I
        is_isolated = rand(model.rng, Categorical([isolated,1-isolated])) == 1 ? true : false 
        mass = is_isolated ? Inf : 1.0
        vel = is_isolated ? (0,0) : sincos(2pi* rand(model.rng,)) .* speed
        hygiene = (hygiene_max-hygiene_min)*rand(model.rng) + hygiene_min
        add_agent!(pos,model,vel, mass, 0, status, 0, hygiene)
    end
    return model
end

function transmit!(a,b,reinfection,model)
    if  count(a.status == :I for a in (a,b)) in (0,2)
        return
    end
    
    infected, healthy = a.status == :I ? (a,b) : (b,a)
    rand(model.rng) < (infected.hygiene + healthy.hygiene)/2 && return 

   
    if healthy.status == :R || healthy.status == :V
        if rand(model.rng) > reinfection 
            return
        end
    end

    healthy.status = :I
end

function vaccine!(a, model)
    a.status = :V   
end

function model_step!(model)
    r = model.interaction_radius
    for (a, b) in interacting_pairs(model, r, :nearest)
        transmit!(a,b,model.reinfection_probability,model)
        elastic_collision!(a, b, :mass)
    end
    
    if model.symulation_time % model.steps_per_day == 0
        for _ in 1:model.vaccine_per_day
            a = random_agent(model, if_susceptible)
            if a != nothing
                vaccine!(a, model)
            else
                model.symulation_time += 1
                return model
            end
        end
       
    end
    model.symulation_time += 1
    return model
end

function quarantine!(a,model)
    if a.status == :I && a.days_infected >= model.detection_time && rand(model.rng) < model.chance_to_go_quarantine 
        a.status = :Q
        a.vel = (0,0)
        a.mass = Inf

    end
end

function quarantine_end!(a,model)
    if a.days_infected == model.infection_period
        recover_or_die!(a,model) 
        a.vel = sincos(2pi* rand(model.rng,)) .* model.speed
        a.mass = 1.0
    end
    return model
end

function recover_or_die!(agent, model)
    if agent.days_infected >= model.infection_period
        if rand(model.rng) <= model.death_rate
#             kill_agent!(agent, model)
        else 
            agent.status = :R
            agent.days_infected = 0
        end
    end
    return model
end

function agent_step!(agent,model)
    move_agent!(agent, model, model.dt)
    agent.status in (:I,:Q) ? agent.days_infected +=1 : agent.days_infected = 0
    agent.status == :Q ? agent.days_quarantine +=1 : agent.days_quarantine = 0
    recover_or_die!(agent, model)
    quarantine!(agent,model)
    quarantine_end!(agent,model)
end
sir_colors(a) = a.status == :S ? "#000000" : a.status == :I ? "#ff0000" : a.status == :Q ? "#00FFFF" : a.status == :V ? "#993399" : "#00FF00"


sir_colors (generic function with 1 method)

In [21]:
import CairoMakie
sir_model = symulation(isolated = 0 )

abm_video("data\\symulation_without_isolation.mp4",
    sir_model,
    agent_step!,
    model_step!,
    title = " Symulation",
    ac = sir_colors,
    frames = 500 , spf = 2, framerate = 25)



# Pokaz filmów

In [3]:
using Base64
function display_mp4(filename)
    display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
    base64encode(open(read,filename)),"""" type="video/mp4"></video>"""))
end

display_mp4 (generic function with 1 method)

In [4]:
display_mp4("data\\symulation_without_isolation.mp4")

LoadError: SystemError: opening file "data\\symulation_without_isolation.mp4": No such file or directory

In [35]:
using JLD2
#an aggregating functions
susceptible(x) = count(i == :S for i in x)
recovered(x) = count(i == :R for i in x)
infected(x) = count(i in [:I,:Q] for i in x)

sir_model1 = symulation(isolated = 0.8)
sir_model2 = symulation(isolated = 1)
sir_model3 = symulation(isolated = 0.4)
sir_model4 = symulation(isolated = 0)

adata = [(:status, susceptible), (:status, recovered), (:status, infected)]
Threads.@spawn global   agents1_df, = run!(sir_model1, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents2_df, = run!(sir_model2, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents3_df, = run!(sir_model3, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents4_df, = run!(sir_model4, agent_step!, model_step!, 1000; adata)


sir_modela = symulation(isolated = 0.3, hygiene_max = 0.8, hygiene_min = 0.7)
sir_modelb = symulation(isolated = 0.3, hygiene_max = 0.8, hygiene_min = 0.5)
sir_modelc = symulation(isolated = 0.3, hygiene_max = 0.8, hygiene_min = 0.3)
sir_modeld = symulation(isolated = 0.3, hygiene_max = 0.8, hygiene_min = 0.0)

Threads.@spawn global   agentsa, = run!(sir_modela, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsb, = run!(sir_modelb, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsc, = run!(sir_modelc, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsd, = run!(sir_modeld, agent_step!, model_step!, 1000; adata)


sir_model1a = symulation(isolated = 0.3, hygiene_max = 1.0,hygiene_min = 0.1)
sir_model1b = symulation(isolated = 0.3, hygiene_max = 0.8,hygiene_min = 0.1)
sir_model1c = symulation(isolated = 0.3, hygiene_max = 0.5,hygiene_min = 0.1)
sir_model1d = symulation(isolated = 0.3, hygiene_max = 0.2,hygiene_min = 0.1)

Threads.@spawn global   agents1a, = run!(sir_model1a, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1b, = run!(sir_model1b, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1c, = run!(sir_model1c, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1d, = run!(sir_model1d, agent_step!, model_step!, 1000; adata)


sir_modelk1 = symulation(chance_to_go_quarantine = 0, isolated = 0.6)
sir_modelk2 = symulation(chance_to_go_quarantine = 12, isolated = 0.6)

Threads.@spawn global   agentsk1, = run!(sir_modelk1, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsk2, = run!(sir_modelk2, agent_step!, model_step!, 1000; adata)


vac_zero = symulation(vaccine_per_day=0)
vac_3 = symulation(vaccine_per_day=3)
vac_6 = symulation(vaccine_per_day=6)
vac_9 = symulation(vaccine_per_day=9)

Threads.@spawn global   model_vac_zero, = run!(vac_zero, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_vac_3, = run!(vac_3, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_vac_6, = run!(vac_6, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_vac_9, = run!(vac_9, agent_step!, model_step!, 1000; adata)


density_model_1000 = symulation(N = 1000)
density_model_800 = symulation(N = 800)
density_model_600 = symulation(N = 600)
density_model_400 = symulation(N = 400)
density_model_200 = symulation(N = 200)

Threads.@spawn global   density_1000, = run!(density_model_1000, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_800, = run!(density_model_800, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_600, = run!(density_model_600, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_400, = run!(density_model_400, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_200, = run!(density_model_200, agent_step!, model_step!, 1000; adata)


model_day_activite_13= symulation(steps_per_day = 13)
model_day_activite_11 = symulation(steps_per_day = 11)
model_day_activite_9 = symulation(steps_per_day = 9)
model_day_activite_7 = symulation(steps_per_day = 7)
model_day_activite_5 = symulation(steps_per_day = 5)

Threads.@spawn global   symulation_day_activite_13, = run!(model_day_activite_13, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   symulation_day_activite_11, = run!(model_day_activite_11, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   symulation_day_activite_9, = run!(model_day_activite_9, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   symulation_day_activite_7, = run!(model_day_activite_7, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   symulation_day_activite_5, = run!(model_day_activite_5, agent_step!, model_step!, 1000; adata)


sir_model_0 = symulation(vaccine_per_day=0, chance_to_go_quarantine=0, isolated=0, hygiene_min = 0.2)
sir_model_1 = symulation(vaccine_per_day=0, chance_to_go_quarantine=0, isolated=0.5, hygiene_min = 0.2)
sir_model_2 = symulation(vaccine_per_day=0, chance_to_go_quarantine=1, isolated=0.5, hygiene_min = 0.3)
sir_model_3 = symulation(vaccine_per_day=2, chance_to_go_quarantine=1, isolated=0.5, hygiene_min = 0.3)
sir_model_4 = symulation(vaccine_per_day=2, chance_to_go_quarantine=1, isolated=0.7,hygiene_min = 0.4 )
sir_model_5 = symulation(vaccine_per_day=3, chance_to_go_quarantine=1, isolated=0.7, hygiene_min = 0.5 )

Threads.@spawn global   model_0, = run!(sir_model_0, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_1, = run!(sir_model_1, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_2, = run!(sir_model_2, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_3, = run!(sir_model_3, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_4, = run!(sir_model_4, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   model_5, = run!(sir_model_5, agent_step!, model_step!, 1000; adata)

Task (runnable) @0x000000001a1f7d00

In [36]:
using JLD2, DataFrames , Plots
@save "data\\izolacja.jld2" agents1_df agents2_df agents3_df agents4_df
@save "data\\higiena_min.jld2" agentsa agentsb agentsc agentsd
@save "data\\higiena_max.jld2" agents1a agents1b agents1c agents1d
@save "data\\kwarantanna.jld2" agentsk1 agentsk2 
@save "data\\vac.jld2" model_vac_zero model_vac_3 model_vac_6 model_vac_9
@save "data\\gestosc.jld2" density_1000 density_800 density_600 density_400 density_200
@save "data\\aktywnosc.jld2" symulation_day_activite_11 symulation_day_activite_13 symulation_day_activite_9 symulation_day_activite_7 symulation_day_activite_5
@save "data\\zmiana_parametrow.jld2" model_0 model_1 model_2 model_3 model_4 model_5 

In [5]:
using JLD2, DataFrames , Plots
@load "data\\izolacja.jld2"

plotly()
x = agents3_df.step
Plots.plot(x, agents3_df[:, 4],
    linecolor = "red",
    lw = 1.5,
    xlabel = "time",
    ylabel = "population",
    label = "infected",
    title = "Pandemic development for simulation",
    legend = :outerright)
Plots.plot!(x, agents3_df[:, 3],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agents3_df[:, 2],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible")



┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\izolacja.jld2": No such file or directory
in expression starting at In[5]:2

In [6]:
using JLD2, DataFrames , Plots
@load "data\\izolacja.jld2"
x = agents1_df.step
plotly()
Plots.plot(x, agents2_df[:, 4],
    linecolor = "#00CC00",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "isolated=1.0",
    title = "Impact of isolation on the number of infected",
    legend = :outerright)
Plots.plot!(x, agents1_df[:, 4],
    linecolor = "#FFD700",
    lw = 1.5,
    label = "isolated=0.8")
Plots.plot!(x, agents3_df[:, 4],
    linecolor = "#FF0000",
    lw = 1.5,
    label = "isolated=0.4")
Plots.plot!(x, agents4_df[:, 4],
    linecolor = "#8B0000",
    lw = 1.5,
    label = "isolated=0.0")

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\izolacja.jld2": No such file or directory
in expression starting at In[6]:2

In [7]:
using JLD2, DataFrames , Plots
@load "data\\higiena_min.jld2"

x = agentsd.step
plotly()
Plots.plot(x, agentsd[:, 4],
    linecolor = "#FFD700",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "min=0.0, max=0.8",
    title = "Impact of hygiene on the number of infection",
    legendtitle = "hygiene",
    legend = :left)
Plots.plot!(x, agentsc[:, 4],
    linecolor = "#FF6600",
    lw = 1.5,
    label = "min=0.3, max=0.8")
Plots.plot!(x, agentsb[:, 4],
    linecolor = "#CC0000",
    lw = 1.5,
    label = "min=0.5, max=0.8")
Plots.plot!(x, agentsa[:, 4],
    linecolor = "#660033",
    lw = 1.5,
    label = "min=0.7, max=0.8")



┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\higiena_min.jld2": No such file or directory
in expression starting at In[7]:2

In [8]:
using JLD2, DataFrames , Plots
@load "data\\higiena_max.jld2"

x = agents1d.step
plotly()
Plots.plot(x, agents1d[:, 4],
    linecolor = "#FFD700",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "max=0.2, min=0.1",
    title = "Impact of hygiene on the number of infection",
    legendtitle = "hygiene",
    legend = :left)
Plots.plot!(x, agents1c[:, 4],
    linecolor = "#FF6600",
    lw = 1.5,
    label = "max=0.5, min=0.1")
Plots.plot!(x, agents1b[:, 4],
    linecolor = "#CC0000",
    lw = 1.5,
    label = "max=0.8, min=0.1")
Plots.plot!(x, agents1a[:, 4],
    linecolor = "#660033",
    lw = 1.5,
    label = "max=1.o, min=0.1")

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\higiena_max.jld2": No such file or directory
in expression starting at In[8]:2

In [9]:
using JLD2, DataFrames , Plots
@load "data\\kwarantanna.jld2"
x = agentsk1.step
plotly()
pk = Plots.plot(x, agentsk1[:, 4],
    linecolor = "red",
    lw = 1.5,
    title = "Without quarantine",
#     xlabel = "time",
    ylabel = "population",
    label = "infected",
    ylims = (0,700),)
Plots.plot!(x, agentsk1[:,3],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agentsk1[:, 2],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible")

pk2 = Plots.plot(x, agentsk2[:, 4],
    linecolor = "red",
    lw = 1.5,
    title = "With quarantine",
    xlabel = "time",
    ylabel = "population",
    label = "infected",
    ylims = (0,700),)
Plots.plot!(x, agentsk2[:,3],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agentsk2[:, 2],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible"
    )

Plots.plot(pk, pk2, layout = (2, 1), legend= :false)


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\kwarantanna.jld2": No such file or directory
in expression starting at In[9]:2

In [10]:
using Plots, JLD2, DataFrames
@load "data\\vac.jld2"
plotly()
x = model_vac_zero.step
Plots.plot(x, model_vac_zero[:, 4],
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "without vaccination",
    title = "Impact of vaccination on the number of infected",
    legend =:right)
Plots.plot!(x, model_vac_3[:, 4],
    lw = 1.5,
    label = "3 vaccination per day")
Plots.plot!(x, model_vac_6[:, 4],
    lw = 1.5,
    label = "6 vaccination per day")
Plots.plot!(x, model_vac_9[:, 4],
    lw = 1.5,
    label = "9 vaccination per day")





┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\vac.jld2": No such file or directory
in expression starting at In[10]:2

In [11]:
using JLD2, DataFrames , Plots
@load "data\\izolacja.jld2"
x = agents1_df.step
plotly()

Plots.plot(x, agents2_df[:, 4],
    linecolor = "#00CC00",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "isolated=1.0",
    title = "Impact of isolation on the number of infected",
    legend = :right)
Plots.plot!(x, agents1_df[:, 4],
    linecolor = "#FFD700",
    lw = 1.5,
    label = "isolated=0.8")
Plots.plot!(x, agents3_df[:, 4],
    linecolor = "#FF0000",
    lw = 1.5,
    label = "isolated=0.4")
Plots.plot!(x, agents4_df[:, 4],
    linecolor = "#8B0000",
    lw = 1.5,
    label = "isolated=0.0")

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\izolacja.jld2": No such file or directory
in expression starting at In[11]:2

In [12]:
using JLD2, DataFrames , Plots
@load "data\\gestosc.jld2"
x = density_1000.step
plotly()
Plots.plot(x, density_1000[:, 4] / sum(density_1000[1,2:4]),
    ylabel="Part of population which is infected",
    xlabel="time",
    label="1000 Population")
Plots.plot!(x, density_800[:, 4] / sum(density_800[1,2:4]),label="800 Population")
Plots.plot!(x, density_600[:, 4] / sum(density_600[1,2:4]),label="600 Population")
Plots.plot!(x, density_400[:, 4] / sum(density_400[1,2:4]),label="400 Population")
Plots.plot!(x, density_200[:, 4] / sum(density_200[1,2:4]),label="200 Population")

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\gestosc.jld2": No such file or directory
in expression starting at In[12]:2

In [13]:
using JLD2, DataFrames, Plots
@load "data\\zmiana_parametrow.jld2"


x = model_0.step

plotly()
Plots.plot(x, model_0[:, 4],
    linecolor = :red,
    lw = 1.5,
    ylabel = "Population which are infected",
    xlabel = "time",
    label = "without parameters",
    title = "Symulation",
    legend = :best)
Plots.plot!(x, model_1[:, 4],
    linecolor = :orange,
    lw = 1.5,
    label = "add isolation")
Plots.plot!(x, model_2[:, 4],
    linecolor = :cyan,
    lw = 1.5,
    label = "add quarantine , increase hygine")
Plots.plot!(x, model_3[:, 4],
    linecolor = :green,
    lw = 1.5,
    label = "add vaccine")
Plots.plot!(x, model_4[:, 4],
    linecolor = :blue,
    lw = 1.5,
    label = "increase isolation and hygine")
Plots.plot!(x, model_5[:, 4],
    linecolor = :purple,
    lw = 1.5,
    label = "increase vaccine and hygine")

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\mkarc\.julia\packages\JLD2\fAkkN\src\JLD2.jl:230


LoadError: LoadError: SystemError: opening file "data\\zmiana_parametrow.jld2": No such file or directory
in expression starting at In[13]:2